In [49]:
# note: same thing for REPL
# note: we use this instead of magic because `black` will otherwise fail to format
#
# Enable autoreload to automatically reload modules when they change

from IPython import get_ipython

# do this so that formatter not messed up
ipython = get_ipython()
ipython.run_line_magic("load_ext", "autoreload")
ipython.run_line_magic("autoreload", "2")

# Import commonly used libraries
import numpy as np
import pandas as pd

# more itertools
import more_itertools as mi

# itertools
import itertools
import collections

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [85]:
import textwrap
import pathlib
import functools

import inspect_explorer

import openai

from inspect_explorer import (
    in_context_tool_use,
    affordances,
    tokenization,
    model_ids,
)

import inspect_explorer.affordances.aider_cli

from inspect_explorer.conversation_manager import (
    ConversationManager,
    ConversationId,
    show_conversation,
)


def suggest_feedback_to_user_about_provided_or_proposed_tools(message: str) -> None:
    print(message)
    raise ValueError("Model provided feedback")


model_workspace_dir = pathlib.Path.cwd() / "model_workspace"


TOOL_DEFINITIONS_WITH_CALLABLES: list[in_context_tool_use.ToolDefinitionWithCallable] = [
    in_context_tool_use.ToolDefinitionWithCallable(
        tool_definition=in_context_tool_use.ToolDefinition(
            function_name="aider_use_specialized_coder_model_to_make_requested_code_edit",
            description=textwrap.dedent(
                """
            Request that another model which specializes in targeted code edits to make the requested
            code change.
            
            While you have the ability to entirely replace files, it's often easier to ask another
            model which is specifically trained to make code changes to make those changes inline.

            The arguments are:
            - read_filepath: The entire contents of `read_filepath` are added to the coder model's context
            - edit_filepath: This is the file you wish to edit
            - message_to_coder_model: Message to send to the coder model
            
            For example:
                                        
            > aider --read inspect_explorer/conversation_manager.py --file tests/test_conversation_manager.py --yes --message 'Write unit tests for conversation manager that can be run with pytest'

            """
            ),
            arguments=[
                "read_filepath",
                "edit_filepath",
                "message_to_coder_model",
            ],
            return_value="Output of the `aider` tool after it has performed the requested change.",
        ),
        callable_fn=affordances.aider_cli.send_aider_message,
    ),
    in_context_tool_use.ToolDefinitionWithCallable(
        tool_definition=in_context_tool_use.ToolDefinition(
            function_name="aider_show_repo_map",
            description=textwrap.dedent(
                """
            Show a map of the current repo, including files and function definitions. 
            
            This is primarily a tool used by IDEs, but is frequently useful to get an
            understanding of a codebase, especially after actions modifying the codebase.
            
            This generates a (default 1024 token) summary of the current repo.

            Example:

                inspect_explorer/in_context_tool_use/initial_explanation.py:
                │def get_in_context_tool_use_initial_explanation() -> str:
                ⋮...

                inspect_explorer/in_context_tool_use/parsing.py:
                ⋮...
                │def parse_tool_use_request_from_model_response(
                │    client: openai.OpenAI,
                │    model_response: openai.types.chat.ChatCompletionMessageParam,
                ⋮...

                inspect_explorer/in_context_tool_use/tool_use_types.py:
                ⋮...
                │class ToolDefinition(pydantic.BaseModel):
                ⋮...
                │class ToolUseRequest(pydantic.BaseModel):
                ⋮...
                │class ToolUseResponse(pydantic.BaseModel):
                ⋮...
                │@dataclasses.dataclass(frozen=True)
                │class ToolDefinitionWithCallable[R, **P]:
                ⋮...
            """
            ),
            arguments=[],
            return_value="String representation of the repo map",
        ),
        callable_fn=affordances.aider_cli.show_repo_map,
    ),
    in_context_tool_use.ToolDefinitionWithCallable(
        tool_definition=in_context_tool_use.ToolDefinition(
            function_name="suggest_feedback_to_user_about_provided_or_proposed_tools",
            description=textwrap.dedent(
                """
            Suggest feedback to the user about the provided or proposed tools. This can be used to request new tools if they'd be easy to implement, as well as changes to existing provided tools.
            """
            ),
            arguments=["message"],
            return_value="None, the user will review the feedback and potentially update tools.",
        ),
        callable_fn=suggest_feedback_to_user_about_provided_or_proposed_tools,
    ),
    in_context_tool_use.ToolDefinitionWithCallable(
        tool_definition=in_context_tool_use.ToolDefinition(
            function_name="execute_bash_command",
            description=textwrap.dedent(
                f"""
            Execute an arbitrary bash command inside the model workspace: `{model_workspace_dir}`
            
            The user will manually confirm the command before it's executed.
            """
            ),
            arguments=["command"],
            return_value=textwrap.dedent(
                """
            The output will be provided as a json string, containing:
                - command: The bash command that was run.
                - exit_code: The exit code of the command.
                - output: The standard output and error of the command.
            """
            ),
        ),
        callable_fn=functools.partial(
            inspect_explorer.affordances.raw_bash_execution.execute_bash_command,
            current_working_directory=model_workspace_dir,
        ),
    ),
]

# create function call manager which requires user confirmation
function_call_manager = in_context_tool_use.FunctionCallManager(
    tool_definitions_with_callables=TOOL_DEFINITIONS_WITH_CALLABLES,
    require_user_confirmation=True,
)

# create conversation manager
# conv_manager = conversation_manager.ConversationManager()

In [72]:
# TODO(bschoen): One function which does this
import json

task_description = """
== BEGIN TASK DESCRIPTION ==

You are an early version of an automated research assistant. Your task currently is to help build out the repo implementing the assistant, which is still in it's early stages.

== END TASK DESCRIPTION == 

"""

# create initial prompt
prompt = task_description + in_context_tool_use.get_in_context_tool_use_initial_explanation()

# add tool definitions for our specific case
tool_definition_list = function_call_manager.get_tool_definition_list()
tool_definition_list_json = json.dumps(
    {
        in_context_tool_use.Constants.TOOL_DEFINITIONS_KEY: [
            x.model_dump() for x in tool_definition_list
        ]
    },
    indent=2,
)

prompt += "\n\nFor your task, the available tools are:\n\n" + tool_definition_list_json


# again repeat the task description
prompt += f"\n\nRemember:\n\n{task_description}"


estimated_token_count = tokenization.estimate_token_count(prompt)

print(f"Estimated token count: {estimated_token_count}")

print(prompt)

Estimated token count: 1251

== BEGIN TASK DESCRIPTION ==

You are an early version of an automated research assistant. Your task currently is to help build out the repo implementing the assistant, which is still in it's early stages.

== END TASK DESCRIPTION == 


You will be provided with tools according to the following protocol. Please note that markers such as [user's turn] and [assistant's turn] are not part of the conversation and are only used to separate turns in the conversation.

```
[user's turn]

"Please add 14 and 21"

{
  "tool_definitions": [
    {
      "function_name": "get_wikipedia_page",
      "description": "Returns full content of a given wikipedia page",
      "arguments": ["title_of_wikipedia_page"],
      "return_value": "returns full content of a given wikipedia page as a json string"
    },
    {
      "function_name": "add",
      "description": "Add two numbers together",
      "arguments": ["first_number", "second_number"],
      "return_value": "returns 

In [ ]:
client = openai.OpenAI()

conversation_manager = ConversationManager()

In [73]:
from typing import Callable

UserMessageContent = str
ResponseContentString = str


def step_conversation_with_user_message(
    client: openai.OpenAI,
    conversation_manager: ConversationManager,
    conversation_id: ConversationId,
    user_message_content: UserMessageContent,
    model_id: model_ids.ModelID = model_ids.ModelID.O1_MINI,
    max_completion_tokens: int = 16000,
) -> ResponseContentString:
    """Step the conversation, taking care of updating state via conversation_manager."""

    # create user message
    user_message: openai.types.chat.ChatCompletionMessageParam = {
        "role": "user",
        "content": user_message_content,
    }

    # add user message to message history
    conversation_manager.add_conversation_message(conversation_id, user_message)

    # retrieve history to send to client
    messages = conversation_manager.get_conversation_messages(conversation_id)

    response = client.chat.completions.create(
        model=model_ids.remove_provider_prefix(model_id),
        messages=messages,
        max_completion_tokens=max_completion_tokens,
    )

    response_content_string: str | None = response.choices[0].message.content

    if not response_content_string:
        raise ValueError(
            f"No response content string. Full response: {response.model_dump_json(indent=4)}"
        )

    # note: using dict representation for consistency with user_message + it's what API expects
    response_message: openai.types.chat.ChatCompletionMessageParam = {
        "role": "assistant",
        "content": response_content_string,
    }

    # add response to conversation
    conversation_manager.add_conversation_message(conversation_id, response_message)

    return response_content_string

    # # parse the tool use request from the response (if any)
    # tool_use_request = in_context_tool_use.parse_tool_use_request_from_model_response(
    #     client=client,
    #     model_response=response,
    # )

    # if not tool_use_request:
    #     print("Model response:")
    #     print(response_message['content'])
    #     return

    # otherwise, we had a tool use request, which we can execute


def handle_tool_use_request_if_present(
    client: openai.OpenAI,
    conversation_manager: ConversationManager,
    conversation_id: ConversationId,
    function_call_manager: in_context_tool_use.FunctionCallManager,
    model_response: ResponseContentString,
) -> ResponseContentString | None:
    # have a smaller model try to parse out a tool use request (returns `None` if none present)
    tool_use_request = in_context_tool_use.parse_tool_use_request_from_model_response(
        client=client,
        model_response=model_response,
    )

    if not tool_use_request:
        print("No tool use request found in model response")
        return None

    # note: this ordering ensures that if the function call fails with exception, we currently
    #       don't add the tool use response to the conversation history (so can start
    #       over from here)
    tool_use_response = function_call_manager.execute_tool(request=tool_use_request)

    # add tool use response to conversation
    tool_use_response_json = json.dumps(
        {in_context_tool_use.Constants.TOOL_USE_RESPONSE_KEY: tool_use_response.model_dump()},
        indent=4,
    )

    # step the conversation with the tool use response as our response
    return step_conversation_with_user_message(
        client=client,
        conversation_manager=conversation_manager,
        conversation_id=conversation_id,
        user_message_content=tool_use_response_json,
    )


# note: can comment this out to keep going with an existing conversation
# conversation_id = conversation_manager.create_new_conversation()
conversation_id = "2024-09-29_12:48_PM__athletic_orchid_carp_of_education"

print(f"Conversation ID: {conversation_id}")

# create bound functions for easier use

# note: type annotations here are critical for tracking arguments
step_conversation_with_user_message_fn: Callable[[UserMessageContent], ResponseContentString] = (
    functools.partial(
        step_conversation_with_user_message,
        client=client,
        conversation_manager=conversation_manager,
        conversation_id=conversation_id,
    )
)

handle_tool_use_request_if_present_fn: Callable[
    [ResponseContentString], ResponseContentString | None
] = functools.partial(
    handle_tool_use_request_if_present,
    client=client,
    conversation_manager=conversation_manager,
    conversation_id=conversation_id,
    function_call_manager=function_call_manager,
)

Conversation ID: 2024-09-29_12:48_PM__athletic_orchid_carp_of_education


In [74]:
response_content_string = step_conversation_with_user_message_fn(user_message_content=prompt)

print(response_content_string)

Loading conversation from: /Users/bronsonschoen/inspect_explorer/conversations/2024-09-29_12:48_PM__athletic_orchid_carp_of_education.json
```json
{
  "assistant_tool_use_request": {
    "function_name": "aider_show_repo_map",
    "arguments": {}
  }
}
```


In [82]:
response_content_string = handle_tool_use_request_if_present_fn(
    model_response=response_content_string
)

print(response_content_string)

Executing tool: function_name='aider_use_specialized_coder_model_to_make_requested_code_edit' 
arguments={'read_filepath': 'inspect_explorer/affordances/aider_cli.py', 'edit_filepath': 
'inspect_explorer/affordances/aider_cli.py', 'message_to_coder_model': "Please add the 'inspect_ai' directory to 
the repo map."}

---
ToolUseRequest: (function_name: aider_use_specialized_coder_model_to_make_requested_code_edit)
---
read_filepath


inspect_explorer/affordances/aider_cli.py

---
edit_filepath


inspect_explorer/affordances/aider_cli.py

---
message_to_coder_model


Please add the 'inspect_ai' directory to the repo map.

---
Please respond with 'y' or 'n'.


Exception: User declined to execute tool

In [86]:
show_conversation(conversation_manager=conversation_manager, conversation_id=conversation_id)

Loading conversation from: /Users/bronsonschoen/inspect_explorer/conversations/2024-09-29_12:48_PM__athletic_orchid_carp_of_education.json


                   Conversation History: 2024-09-29_12:48_PM__athletic_orchid_carp_of_education                    
┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ role      ┃ content                                                                                             ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ user      │ == BEGIN TASK DESCRIPTION ==                                                                        │
│           │                                                                                                     │
│           │ You are an early version of an automated research assistant. Your task currently is to help build   │
│           │ out the repo implementing the assistant, which is still in it's early stages.                       │
│           │                                                                                                     │
│           │ == END TASK DESCRIPTION ==                                                                          │
│           │                                                                                                     │
│           │ You will be provided with tools according to the following protocol. Please note that markers such  │
│           │ as [user's turn] and [assistant's turn] are not part of the conversation and are only used to       │
│           │ separate turns in the conversation.                                                                 │
│           │                                                                                                     │
│           │                                                                                                     │
│           │  [user's turn]                                                                                      │
│           │                                                                                                     │
│           │  "Please add 14 and 21"                                                                             │
│           │                                                                                                     │
│           │  {                                                                                                  │
│           │    "tool_definitions": [                                                                            │
│           │      {                                                                                              │
│           │        "function_name": "get_wikipedia_page",                                                       │
│           │        "description": "Returns full content of a given wikipedia page",                             │
│           │        "arguments": ["title_of_wikipedia_page"],                                                    │
│           │        "return_value": "returns full content of a given wikipedia page as a json string"            │
│           │      },                                                                                             │
│           │      {                                                                                              │
│           │        "function_name": "add",                                                                      │
│           │        "description": "Add two numbers together",                                                   │
│           │        "arguments": ["first_number", "second_number"],                                              │
│           │        "return_value": "returns the numeric value of adding together `first_number` and             │
│           │  `second_number`"                                                                                   │
│           │      }                                    